In [ ]:

import pandas as pd
import json

# 讀取 JSON 檔案
with open('龜山區.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# 轉為 DataFrame
df = pd.DataFrame(data)

# 顯示表格
print(df)

JSONDecodeError: Unterminated string starting at: line 1 column 23515053 (char 23515052)

In [ ]:
with open('龜山區.json', 'r', encoding='utf-8') as file:
    content = file.read()

# 嘗試手動補右中括號
if not content.strip().endswith(']'):
    content += ']'

data = json.loads(content)
df = pd.DataFrame(data)
print(df)

      AA11    AA12                                   a              b    bn  \
0       其他                       陳厝段00810000地號#陳厝段81地號                        
1       其他                      公西段08720000地號#公西段872地號                        
2       其他                     楓義段14550000地號#楓義段1455地號                        
3        商               龜山區中山街００１８號地下一層#龜山區中山街１８號地下一層                        
4                              龜山區文昌五街００４２號#龜山區文昌五街４２號  華廈(10層含以下有電梯)  A8優品   
...    ...     ...                                 ...            ...   ...   
54654    農          牛角坡段樟腦寮小段02070000地號#牛角坡段樟腦寮小段207地號                        
54655       丙種建築用地              東嶺段09420000地號#東嶺段942地號                        
54656    住                        善捷段00020000地號#善捷段2地號                        
54657    住             龜山區自強南路２８７巷００４５號#龜山區自強南路２８７巷４５號            透天厝         
54658    住              龜山區明成街０２１６之６號六樓#龜山區明成街２１６之６號六樓   套房(1房(1廳)1衛)         

           bs city                                 

In [ ]:
import csv
import json

input_csv = "taoyuan_cleaned_no.csv"     # ← 把這裡改成你的檔名
output_json = "taoyuan_cleaned_no.json"      # ← 輸出的 JSON 檔名

with open(input_csv, mode='r', encoding='utf-8') as csv_file, \
     open(output_json, mode='w', encoding='utf-8') as json_file:

    reader = csv.DictReader(csv_file)

    json_file.write('[\n')  # 開頭

    first = True
    for row in reader:
        if not first:
            json_file.write(',\n')
        json.dump(row, json_file, ensure_ascii=False)
        first = False

    json_file.write('\n]')  # 結尾

print("✅ 轉換完成！JSON 檔案已儲存為：", output_json)


✅ 轉換完成！JSON 檔案已儲存為： taoyuan_cleaned_no.json


In [ ]:
#清洗
import json
import re

input_file = 'taoyuan_cleaned_no -by MaxAI.json'           # 改成你的來源檔案
output_file = 'cleaned_output.jsonl'     # BigQuery 能吃的 JSONL 檔案

def clean_field_name(name):
    # 移除不合法字元，只保留中英文字母、數字與底線
    name = re.sub(r'[^\w\u4e00-\u9fff]', '', name)
    return name

with open(input_file, 'r', encoding='utf-8') as f:
    data = json.load(f)  # 假設原始是 JSON 陣列

with open(output_file, 'w', encoding='utf-8') as f:
    for record in data:
        cleaned = {clean_field_name(k): v for k, v in record.items()}
        json.dump(cleaned, f, ensure_ascii=False)
        f.write('\n')

print("✅ 欄位名稱清理完成，輸出檔案：", output_file)


✅ 欄位名稱清理完成，輸出檔案： cleaned_output.jsonl


In [ ]:
#清理空白欄位名稱
import pandas as pd
import json
import re

# 讀取 CSV
df = pd.read_csv('taoyuan_cleaned_no.csv')  # 請改成你的檔案名稱

# 清理欄位名稱
def clean_column(col):
    if not isinstance(col, str) or col.strip() == '':
        return None  # 丟掉空白欄位名
    # 僅保留中英文、數字、底線
    cleaned = re.sub(r'[^\w\u4e00-\u9fff]', '', col)
    return cleaned if cleaned else None  # 若清完還是空，就不要保留

# 套用欄位清理
df.columns = [clean_column(col) for col in df.columns]

# 移除欄位名為 None 的欄位
df = df.loc[:, df.columns.notnull()]

# 輸出成 newline-delimited JSON
with open('output_cleaned.json2', 'w', encoding='utf-8') as f:
    for record in df.to_dict(orient='records'):
        json.dump(record, f, ensure_ascii=False)
        f.write('\n')

print("✅ 檔案處理完成，已儲存為 output_cleaned.json2")


/tmp/ipython-input-2720534025.py:7: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('taoyuan_cleaned_no.csv')  # 請改成你的檔案名稱


✅ 檔案處理完成，已儲存為 output_cleaned.json2


In [ ]:
#掃描並修復空欄位名稱的 JSON 檔案
import json
import os

input_file = 'output_cleaned.json'          # 請換成你的 JSONL 檔名
output_file = 'cleaned_file.json'

fixed_count = 0

with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
    for i, line in enumerate(infile, 1):
        try:
            record = json.loads(line)
            new_record = {}
            empty_field_counter = 1
            for key, value in record.items():
                new_key = key if key.strip() else f'field_{empty_field_counter}'
                if new_key != key:
                    fixed_count += 1
                    empty_field_counter += 1
                new_record[new_key] = value
            outfile.write(json.dumps(new_record, ensure_ascii=False) + '\n')
        except json.JSONDecodeError:
            print(f"⚠️ 第 {i} 行 JSON 無法解析，已略過。")

print(f"✅ 清理完成！已修復 {fixed_count} 個空欄位名稱。結果輸出到：{output_file}")


✅ 清理完成！已修復 0 個空欄位名稱。結果輸出到：cleaned_file.json


In [ ]:
#轉換到bq可對應的欄位

import pandas as pd

df = pd.read_csv("/content/taoyuan_cleaned_no.csv")

print("原始欄位:", df.columns.tolist())

df.rename(columns={
    "transaction number": "transaction_number",
    "transaction date": "transaction_date",
    "transaction target": "transaction_target",
    "land area sqm": "land_area_sqm",
    "urban zone": "urban_zone",
    "non urban zone": "non_urban_zone",
    "non urban usage": "non_urban_usage",
    "transaction date": "transaction_date",
    "transaction count": "transaction_count",
    "floor level": "floor_level",
    "total floors": "total_floors",
    "building type": "building_type",
    "main purpose": "main_purpose",
    "main material": "main_material",
    "building completion date": "building_completion_date",
    "building area sqm": "building_area_sqm",
    "layout rooms": "layout_rooms",
    "layout living rooms": "layout_living_rooms",
    "layout bathrooms": "layout_bathrooms",
    "layout partitions": "layout_partitions",
    "has management org": "has_management_org",
    "total price": "total_price",
    "price per sqm": "price_per_sqm",
    "parking type": "parking_type",
    "parking area sqm": "parking_area_sqm",
    "parking total price": "parking_total_price",
    "remark": "remark",
    "serial no": "serial_no",
    "main building area": "main_building_area",
    "auxiliary building area": "auxiliary_building_area",
    "balcony area": "balcony_area",
    "has elevator": "has_elevator",
    "transaction serial no": "transaction_serial_no",
    "parking_area_sqm_alt": "parking_area_sqm_alt",
    "parking_total_price_alt": "parking_total_price_alt",
    "parking_price_per_sqm": "parking_price_per_sqm",
    }, inplace=True)

print("改名後欄位:", df.columns.tolist())

df["transaction_date"] = pd.to_datetime(df["transaction_date"]).dt.date


/tmp/ipython-input-35461283.py:5: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/taoyuan_cleaned_no.csv")


原始欄位: ['transaction number', 'district', 'transaction_target', 'address', 'land_area_sqm', 'urban_zone', 'non_urban_zone', 'non_urban_usage', 'transaction_date', 'transaction_count', 'floor_level', 'total_floors', 'building_type', 'main_purpose', 'main_material', 'building_completion_date', 'building_area_sqm', 'layout_rooms', 'layout_living_rooms', 'layout_bathrooms', 'layout_partitions', 'has_management_org', 'total_price', 'price_per_sqm', 'parking_type', 'parking_area_sqm', 'parking_total_price', 'remark', 'serial_no', 'main_building_area', 'auxiliary_building_area', 'balcony_area', 'has_elevator', 'transaction_serial_no', 'parking_area_sqm_alt']
改名後欄位: ['transaction_number', 'district', 'transaction_target', 'address', 'land_area_sqm', 'urban_zone', 'non_urban_zone', 'non_urban_usage', 'transaction_date', 'transaction_count', 'floor_level', 'total_floors', 'building_type', 'main_purpose', 'main_material', 'building_completion_date', 'building_area_sqm', 'layout_rooms', 'layout_liv

In [ ]:
import pandas as pd

df = pd.read_csv("/content/taoyuan_cleaned_no.csv")

print("原始欄位:", df.columns.tolist())

df.rename(columns={
    "transaction number": "transaction_number",
    "transaction date": "transaction_date",
    "transaction target": "transaction_target",
    "address": "address",
    "land area sqm": "land_area_sqm",
    "urban zone": "urban_zone",
    "non urban zone": "non_urban_zone",
    "non urban usage": "non_urban_usage",
    "transaction count": "transaction_count",
    "floor level": "floor_level",
    "total floors": "total_floors",
    "building type": "building_type",
    "main purpose": "main_purpose",
    "main material": "main_material",
    "building completion date": "building_completion_date",
    "building area sqm": "building_area_sqm",
    "layout rooms": "layout_rooms",
    "layout living rooms": "layout_living_rooms",
    "layout bathrooms": "layout_bathrooms",
    "layout partitions": "layout_partitions",
    "has management org": "has_management_org",
    "total price": "total_price",
    "price per sqm": "price_per_sqm",
    "parking type": "parking_type",
    "parking area sqm": "parking_area_sqm",
    "parking total price": "parking_total_price",
    "remark": "remark",
    "serial no": "serial_no",
    "main building area": "main_building_area",
    "auxiliary building area": "auxiliary_building_area",
    "balcony area": "balcony_area",
    "has elevator": "has_elevator",
    "transaction serial no": "transaction_serial_no",
    "parking area sqm alt": "parking_area_sqm_alt",
    "parking total price alt": "parking_total_price_alt",
    "parking price per sqm": "parking_price_per_sqm",
}, inplace=True)

print("改名後欄位:", df.columns.tolist())

df["transaction_date"] = pd.to_datetime(df["transaction_date"]).dt.date

# 存成新檔，方便之後直接上傳
df.to_csv("/content/taoyuan_cleaned_no_renamed.csv", index=False)

print("欄位改名並存檔完成！")


/tmp/ipython-input-2846865960.py:3: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/taoyuan_cleaned_no.csv")


原始欄位: ['transaction number', 'district', 'transaction_target', 'address', 'land_area_sqm', 'urban_zone', 'non_urban_zone', 'non_urban_usage', 'transaction_date', 'transaction_count', 'floor_level', 'total_floors', 'building_type', 'main_purpose', 'main_material', 'building_completion_date', 'building_area_sqm', 'layout_rooms', 'layout_living_rooms', 'layout_bathrooms', 'layout_partitions', 'has_management_org', 'total_price', 'price_per_sqm', 'parking_type', 'parking_area_sqm', 'parking_total_price', 'remark', 'serial_no', 'main_building_area', 'auxiliary_building_area', 'balcony_area', 'has_elevator', 'transaction_serial_no', 'parking_area_sqm_alt']
改名後欄位: ['transaction_number', 'district', 'transaction_target', 'address', 'land_area_sqm', 'urban_zone', 'non_urban_zone', 'non_urban_usage', 'transaction_date', 'transaction_count', 'floor_level', 'total_floors', 'building_type', 'main_purpose', 'main_material', 'building_completion_date', 'building_area_sqm', 'layout_rooms', 'layout_liv